In [13]:
import sqlite3
import csv
import pandas as pd
import re

db_path = "sir.db"
covid_paths = "../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-"

conn = sqlite3.connect(db_path)
c = conn.cursor()
c2 = conn.cursor()

c.execute('PRAGMA foreign_keys = ON')
c2.execute('PRAGMA foreign_keys = ON')

In [2]:
# Create tables
create_cities_table_command = '''
CREATE TABLE IF NOT EXISTS cities (
    country VARCHAR(255),    
    city VARCHAR(255),
    location VARCHAR(255) NOT NULL,
    date DATE NOT NULL,
    susceptible INT,
    infected INT,
    delta_infected INT,
    recovered INT,
    PRIMARY KEY(date, location)
    );
'''
c.execute(create_cities_table_command)
conn.commit()

In [4]:
confirmed_file = covid_paths + 'Confirmed.csv'
deaths_file = covid_paths + 'Deaths.csv'
recovered_file = covid_paths + 'Recovered.csv'
df_confirmed = pd.read_csv(confirmed_file)
df_deaths = pd.read_csv(deaths_file)
df_recovered = pd.read_csv(recovered_file)

print(df_confirmed.head())

  Province/State  Country/Region      Lat      Long  1/22/20  1/23/20  \
0          Anhui  Mainland China  31.8257  117.2264        1        9   
1        Beijing  Mainland China  40.1824  116.4142       14       22   
2      Chongqing  Mainland China  30.0572  107.8740        6        9   
3         Fujian  Mainland China  26.0789  117.9874        1        5   
4          Gansu  Mainland China  36.0611  103.8343        0        2   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/26/20  2/27/20  2/28/20  \
0       15       39       60       70  ...      989      989      990   
1       36       41       68       80  ...      400      410      410   
2       27       57       75      110  ...      576      576      576   
3       10       18       35       59  ...      294      296      296   
4        2        4        7       14  ...       91       91       91   

   2/29/20  3/1/20  3/2/20  3/3/20  3/4/20  3/5/20  3/6/20  
0      990     990     990     990     990     990     990  


In [11]:
dates = []
for col in df_confirmed.columns:
    pattern = re.compile('^\d+/\d+/\d+')
    if pattern.match(col):
        dates.append(col)

for row in df_confirmed.iterrows():
    city = row[1]['Province/State']
    country = row[1]['Country/Region']
    location = country
    if (isinstance(city, str)):
         location = location + ', ' + city

    prev_infected = -1
    prev_prev_infected = -1
    prev_date = ""
    for date in dates:
        split_date = date.split('/')
        month = split_date[0].zfill(2)
        day = split_date[1].zfill(2)
        year = '20' + split_date[2]
        date_str = year + '-' + month + '-' + day
        
        cur_infected = row[1][date]
        
        if prev_infected == -1:
            prev_infected = cur_infected
            prev_date = date_str
            continue
            
        if prev_prev_infected == -1:
            prev_prev_infected = prev_infected
            prev_infected = cur_infected
            prev_date = date_str
            continue
        
        delta_infected = (cur_infected - prev_prev_infected) / 2
        
        
        c.execute('UPDATE cities SET delta_infected="'+str(cur_infected - prev_infected)+'" WHERE location="'+location+'" AND date="'+date_str+'"')
        c.execute('UPDATE cities SET delta_infected="'+str(delta_infected)+'" WHERE location="'+location+'" AND date="'+prev_date+'"')
        
        prev_prev_infected = prev_infected
        prev_infected = cur_infected
        prev_date = date_str
        #c.execute(
        #'''INSERT INTO cities VALUES (?, ?, ?, ?, ?, ?, ?, ?);''', 
        #(country, city, location, date_str, None, cur_infected, delta_infected, None))

conn.commit()
        

In [9]:
for row in df_deaths.iterrows():
    city = row[1]['Province/State']
    country = row[1]['Country/Region']
    location = country
    if (isinstance(city, str)):
         location = location + ', ' + city
    
    for date in dates:
        split_date = date.split('/')
        month = split_date[0].zfill(2)
        day = split_date[1].zfill(2)
        year = '20' + split_date[2]
        date_str = year + '-' + month + '-' + day
        
        recovered = row[1][date]
        
        c.execute('UPDATE cities SET recovered="'+str(recovered)+'" WHERE location="'+location+'" AND date="'+date_str+'";')

conn.commit()

In [21]:
for row in df_recovered.iterrows():
    city = row[1]['Province/State']
    country = row[1]['Country/Region']
    location = country
    if (isinstance(city, str)):
         location = location + ', ' + city
    
    
    for date in dates:
        split_date = date.split('/')
        month = split_date[0].zfill(2)
        day = split_date[1].zfill(2)
        year = '20' + split_date[2]
        date_str = year + '-' + month + '-' + day
        
        c.execute('SELECT recovered FROM cities where location="'+location+'" AND date="'+date_str+'" LIMIT 1;')
        
        cur_recovered = 0
        
        for q in c:
            cur_recovered = q[0]
        cur_recovered += row[1][date]
        
        c.execute('UPDATE cities SET recovered="'+str(cur_recovered)+'" WHERE location="'+location+'" AND date="'+date_str+'";')

conn.commit()

In [14]:
c.execute('''SELECT location, date, infected, recovered FROM cities''')

for row in c:
    location = row[0]
    date = row[1]
    infected = row[2]
    recovered = row[3]
    
    updated_infected = infected - recovered
    c2.execute('UPDATE cities SET infected="'+str(updated_infected)+'" WHERE location ="'+location+'" AND date="'+date+'"')
    
conn.commit()

In [15]:
c.execute('''SELECT * FROM cities''')

for row in c:
    print(row)

('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-23', None, 9, 7, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-24', None, 15, 15, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-25', None, 39, 22.5, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-26', None, 60, 15.5, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-27', None, 70, 23, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-28', None, 106, 41, 0)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-29', None, 150, 47, 2)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-30', None, 198, 42.5, 2)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-01-31', None, 234, 48.5, 3)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-02-01', None, 292, 51.5, 5)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-02-02', None, 333, 55.5, 7)
('Mainland China', 'Anhui', 'Mainland China, Anhui', '2020-0

In [46]:
c.execute('''DROP TABLE cities;''')
conn.commit()